# Decision Trees 

Author: Jacob McCabe

## Overview

This notebook will take an in-depth look at decision trees:

1. What is a decision tree?
2. Ensemble methods
3. Comparison of a single decision tree and a random forest

## What is a decision tree?

Decision Trees are family of models for supervised leraning able to be used for classification and regression. While simple, their predictive power does not compete with other supervised learning models. That being said, there are ensemble methods that increase the predictive accuracy of tree-based methods such as boosting, random forests, and Baysian additive regression trees.

#### Motivating example

Consider the `Iris` dataset - there are four features that describe three possible classifications if iris. Suppose we just consider two of the features to classify iris plants: petal width (petal_w) and petal length (petal_l). In a toy decision tree based on these two features, we need a set of splitting rules. For example, we could split observations by assigning `petal_w < 1.2` to the left branch and `petal_w >= 1.2` to the right branch. Then we could further split the right branch by splitting again on `petal_l < 3.76` and `petal_l >= 3.76` giving us three regions. Now we have the predictor space stratified into spaces: plants with a petal width less than 1.2cm, plants with petal width greater than 1.2cm and petal length less than 3.76cm, and plants with petal width greater than 1.2cm and petal length greater than 3.76cm. We can represent them as:
- $R_1 = \{X|petal\_ w<1.2\}$
- $R_2 = \{X|petal\_ w\ge 1.2, petal\_ l<3.76\}$
- $R_3 = \{X|petal\_ w\ge 1.2, petal\_ l\ge 3.76\}$

$R_1$, $R_2$, and $R_3$ are ***leaf nodes*** of the tree, they are end-points where the prediction is made. The ***internal nodes*** in our tree are at  `petal_w >= 1.2` and `petal_l < 3.76`. While this is overly simplified, we can see how the tree takes form and how it is made.

#### Splitting the feature space

The first step in a decision tree is pretty simple, we want to split the predictor space. Split the set of values for our featurez, $X_1,\ldots,X_p$, into distinct and non-overlapping regions, $R_1,\ldots,R_J$. The second step is that for each observation that falls into that region $j$, we make the same prediction. For classification, the prediction is the class most represented in $R_j$. For regression, the prediction is the mean response value in $R_j$.

When it comes to creating our regions, the goal is to minimize the error. We will take this to be ***root sum of squares (RSS)***.     
$RSS = \sum_{j=1}^{J} \sum_{i\in R_{j}} (y_i - \hat{y}_{R_{j}})^2$, where $\hat{y}_{R_{j}}$ is the predicted value of observation $i$ falling in region $j$.

To do this, we can brush off our dynamic programming skills and use a ***top-down greedy approach***. As a quick reminder of greedy algorithms, at each step we make a "greedy decision" based upon a predefined rule that gets us the optimal choice at each step. They are particularly useful when it is to expensive computationally to try every possible option for a problem. In this case, it would be too expensive to try every possible decision boundary, so we want to make an optimal split for each feature in $X_j$. Given feature $X_j$ and a cutpoint $s$ such that splitting on s gives $\{X | X_j<s\}$ and $\{X | X_j\ge s\}$, we pick $s$ such that it minimizes the RSS for that feature. We want to consider every $X_j \in \{X_1, \ldots, X_p \}$ with it's optimal $s$, and pick the $j$ pairing with the smallest RSS. This gives us $R_1$ and $R_2$ and $RSS = RSS_{R_1} + RSS_{R_2}$.

Now we have to repeat the process, but with some nuance. We want to pick the remaining features and its cutpoint to split ***one*** of the regions $R_1$ or $R_2$. That gives us a third region. We proceed with this process of splitting one of the regions such that it gives us the minimum total RSS. Once we have $j$ regions, we're done.

#### Pruning the tree

An issue that can arise from this is that the decision tree is overfitting the training data (high variance). To combat this issue and reduce model complexity, we want to ***prune*** the tree. As with every learned model, we want to find some balance between too large and too small of a model that will adequately capture complexity without overfitting. One strategy is to increase the number of regions until there are only a few observations in each region, and then prune back the nodes that don't provide much benefit. With pruning, we want to be careful to not reduce predictive accuracy on a ***cross-validation set***. 

A common pruning algorithm is ***cost complexity pruning***. While we're not going to dive into the details of this algorithm, the idea is that we consider a series of subtrees that vary in size. Each tree is scored on the cross-validation set and the larger a tree is, the more it is penalized. The end result is that we get an decision tree of optimal size.

## Ensemble methods

As mentioned at the beginning, a single decision tree on its own doesn't have great predictive accuracy. Ensemble methods use several decision trees in order to build a more robust, accurate model. While there are several ensemble methods out there, we will just discuss two: boosted trees and random forests.

#### Boosted trees

With a ***boosted*** model, the goal is to train an ensemble of trees sequentially where each tree is grown using information from previous trees. This helps the model slowly perform better in places it struggled before. In essence, if we are given a current tree, we would train another tree on its residuals instead of the explicit data. This new tree gets added into the fitted function and the residuals get updated.

#### Random forests

***Random forest*** models are very similar to another ensemble method called ***bagging***, but they have a slight improvement. The idea is to build a number of decision trees, with each one trained on a bootstrapped training sample. When building each tree, we want to take a random sample of $m$ features out of $p$ features each time a split is considered (resampling at each split). The split is only able to use one of those $m$, randomly-sampled features. Picking $m \approx \sqrt{p}$ means that many of the features aren't considered, decorrelating many of the trees within the ensemble.

## Resources

- [Pattern Recognition and Machine Learning, Ch. 14](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf)
- [An Introduction to Statistical Learning, Ch. 8](https://hastie.su.domains/ISLR2/ISLRv2_website.pdf)
- [Decision Trees Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning)
- [Decision Tree Pruning Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_pruning)
- [Scikit-learn](https://scikit-learn.org/stable/)

## Comparing a decision tree to random forest

#### Import libraries

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

The data we are using is Concrete Compressive Strength. It contains no null values, 8 continuous features, and the target is Concrete Compressive Strength. We will use it to compare a single decision tree with a random forest. We can expect the random forest to have more predictive accuracy than the single decision tree.

In [28]:
concrete = pd.read_excel('data/Concrete_Data.xls')

new_col_names = {'Cement (component 1)(kg in a m^3 mixture)' : 'comp1',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)' : 'comp2',
       'Fly Ash (component 3)(kg in a m^3 mixture)' : 'comp3',
       'Water  (component 4)(kg in a m^3 mixture)' : 'comp4',
       'Superplasticizer (component 5)(kg in a m^3 mixture)' : 'comp5',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)' : 'comp6',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)' : 'comp7', 
       'Age (day)' : 'age',
       'Concrete compressive strength(MPa, megapascals) ' : 'compressive_strength'}

concrete = concrete.rename(columns=new_col_names)
concrete.describe().T

,count,mean,std,min,25%,50%,75%,max
comp1,1030.0,281.165631,104.507142,102.000000,192.375000,272.900000,350.000000,540.000000
comp2,1030.0,73.895485,86.279104,0.000000,0.000000,22.000000,142.950000,359.400000
comp3,1030.0,54.187136,63.996469,0.000000,0.000000,0.000000,118.270000,200.100000
comp4,1030.0,181.566359,21.355567,121.750000,164.900000,185.000000,192.000000,247.000000
comp5,1030.0,6.203112,5.973492,0.000000,0.000000,6.350000,10.160000,32.200000
comp6,1030.0,972.918592,77.753818,801.000000,932.000000,968.000000,1029.400000,1145.000000
comp7,1030.0,773.578883,80.175427,594.000000,730.950000,779.510000,824.000000,992.600000
age,1030.0,45.662136,63.169912,1.000000,7.000000,28.000000,56.000000,365.000000
compressive_strength,1030.0,35.817836,16.705679,2.331808,23.707115,34.442774,46.136287,82.599225


In [30]:
X = concrete.copy()
y = X.pop("compressive_strength")

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=10)

#### Decision Tree

Lets implement an out-of-the-box instance of the `DecisionTreeRegressor` from `scikit-learn` first. We can see that this model found that the four most important features were comp1, age, comp2, and comp4. The impurity-based feature importance is calculated by the normalized total reduction of the criterion contributed by the feature.

In [36]:
tree_steps = [('scaler', StandardScaler()), ('tree', DecisionTreeRegressor())]
tree_pipe = Pipeline(tree_steps)
tree_pipe.fit(X_train, y_train)

tree_importance = [round(i, 4) for i in list(tree_pipe[1].feature_importances_)]
list(zip(X.columns, tree_importance))

[('comp1', 0.3685),
 ('comp2', 0.1092),
 ('comp3', 0.0085),
 ('comp4', 0.1061),
 ('comp5', 0.0386),
 ('comp6', 0.0278),
 ('comp7', 0.0319),
 ('age', 0.3094)]

#### Random Forest

Now lets implement an out-of-the-box instance of the `RandomForestRegressor`. Looking at the four most important features, we have age, comp1, comp4, and comp5. The difference in these features between the two models is expected becasue of how random forests train many trees and the decorrelating of the trees.

In [39]:
forest_steps = [('scaler', StandardScaler()), ('forest', RandomForestRegressor())]
forest_pipe = Pipeline(forest_steps)
forest_pipe.fit(X_train, y_train)

forest_importance = [round(i, 4) for i in list(forest_pipe[1].feature_importances_)]
list(zip(X.columns, forest_importance))

[('comp1', 0.3195),
 ('comp2', 0.0729),
 ('comp3', 0.0217),
 ('comp4', 0.106),
 ('comp5', 0.0784),
 ('comp6', 0.0259),
 ('comp7', 0.0401),
 ('age', 0.3355)]

In [17]:
tree_pipe.score(X_test, y_test)

0.892977019704554

In [18]:
forest_pipe.score(X_test, y_test)

0.9199698899877466

As expected, the random forest scored better than the single decision tree, having a higher $R^2$ value.